<a href="https://colab.research.google.com/github/chrisqin15/MIE1628/blob/main/MIE1628_A2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MIE1628 Assignment 2

## Initializing Spark

In [2]:
# Install Dependencies
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [3]:
# Set Environment Variables:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [4]:
# all spark imports
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkConf, SparkContext

In [5]:
spark = SparkSession.builder.master("local[*]").appName("A2").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark

## PART A:

### 1. Count the odd and even numbers using file ‘integer.txt’ download it from the Quercus. Show your code and output.

In [5]:
from google.colab import files
uploaded = files.upload()

Saving integer.txt to integer.txt


In [6]:
# Import the integer.txt file
intRDD = spark.sparkContext.textFile('integer.txt')

In [72]:
# Transfer to float form
numRDD = intRDD.map(lambda x: float(x))

In [103]:
# filter the even number and count
evennum = numRDD.filter(lambda x: x % 2 == 0).count()
print('There are', evennum, 'even numbers in the integer.txt file.')

There are 514 even numbers in the integer.txt file.


In [104]:
# filter the odd number and count
oddnum = numRDD.filter(lambda x: x % 2 != 0).count()
print('There are', oddnum, 'odd numbers in the integer.txt file.')

There are 496 odd numbers in the integer.txt file.


### 2. Calculate the salary sum per department using file ‘salary.txt’ download it from the Quercus. Show department name and salary sum. Show your code and output.

In [105]:
from google.colab import files
uploaded = files.upload()

Saving salary.txt to salary.txt


In [122]:
# Import the salary.txt file
salaryRDD = spark.sparkContext.textFile('salary.txt')

In [123]:
# Separate by " "
salaryarrayRDD = salaryRDD.map(lambda x: x.split(" "))

In [124]:
# Transfer the second column to float form
salaryfRDD = salaryarrayRDD.map(lambda x: (x[0], float(x[1])))

In [125]:
# Calculate the salary sum per department
salarysumRDD = salaryfRDD.reduceByKey(lambda x,y: x+y)

In [126]:
salarysumRDD.collect()

[('Sales', 3488491.0),
 ('Research', 3328284.0),
 ('Developer', 3221394.0),
 ('QA', 3360624.0),
 ('Marketing', 3158450.0)]

### 3. Implement MapReduce using Pyspark on file ‘shakespeare.txt’ download it from the Quercus. Show how many times these particular words appear in the document: Shakespeare, When, Lord, Library, GUTENBERG, WILLIAM, COLLEGE and WORLD. (Count exact words only)

In [204]:
from google.colab import files
uploaded = files.upload()

Saving shakespeare-1.txt to shakespeare-1.txt


In [205]:
# Import the shakespeare.txt file
shakespeareRDD = spark.sparkContext.textFile('shakespeare-1.txt')

In [206]:
# Remove the punctuation and separate sentence into words
import re
shakesarrayRDD = shakespeareRDD.flatMap(lambda x: re.split(r'\W+', x))

In [207]:
# Remove all entries where the word is ''
shakespearearrayRDD = shakesarrayRDD.filter(lambda x: x != '')

In [209]:
# Create the particular words list
wordsList = ['Shakespeare', 'When', 'Lord', 'Library', 'GUTENBERG', 
             'WILLIAM', 'COLLEGE', 'WORLD']

In [210]:
# Filter all the particular words in shakespeare.txt
wordsListRDD = shakespearearrayRDD.filter(lambda x: x in wordsList)

In [211]:
def wordCount(wordListRDD):
    return wordListRDD.map(lambda w : (w, 1)).reduceByKey(lambda a, b : a + b)

In [212]:
wordCount(wordsListRDD).collect()

[('Shakespeare', 22),
 ('GUTENBERG', 100),
 ('WILLIAM', 128),
 ('WORLD', 98),
 ('COLLEGE', 98),
 ('When', 406),
 ('Lord', 402),
 ('Library', 5)]

### 4. Calculate top 15 and bottom 15 words using file ‘shakespeare.txt’ download it from the Quercus. Show 15 words with most count and 15 words with least count. You can limit by 15 in ascending and descending order of count. Show your code and output.

In [213]:
# Top 15 words
wordCount(shakespearearrayRDD).takeOrdered(15, key = lambda x: -x[1])

[('the', 11412),
 ('I', 9714),
 ('and', 8942),
 ('of', 7968),
 ('to', 7742),
 ('a', 5796),
 ('you', 5360),
 ('my', 4922),
 ('in', 4803),
 ('d', 4365),
 ('that', 3864),
 ('And', 3735),
 ('is', 3722),
 ('not', 3595),
 ('me', 3448)]

In [214]:
# Bottom 15 words
wordCount(shakespearearrayRDD).takeOrdered(15, key = lambda x: x[1])

[('anyone', 1),
 ('restrictions', 1),
 ('online', 1),
 ('www', 1),
 ('gutenberg', 1),
 ('org', 1),
 ('COPYRIGHTED', 1),
 ('Details', 1),
 ('guidelines', 1),
 ('Posting', 1),
 ('2011', 1),
 ('EBook', 1),
 ('January', 1),
 ('1994', 1),
 ('Character', 1)]

## PART B

### 1. Describe your data. Calculate top 10 movies with highest ratings and top 10 users who provided highest ratings. Show your code and output.

In [6]:
# upload file
from google.colab import files
uploaded = files.upload()

Saving movies.csv to movies.csv


In [7]:
# import file
df = spark.read.csv('movies.csv', header = True, inferSchema = True)

In [8]:
# Count the number of times a movie that is rated and show top 10
df_movie_count = df.groupby('movieId').count()
df_movie_count.sort('count', ascending = False).show(10)

+-------+-----+
|movieId|count|
+-------+-----+
|     51|   20|
|     50|   20|
|     29|   20|
|      6|   20|
|     22|   20|
|     15|   19|
|     55|   19|
|     94|   19|
|     68|   19|
|      2|   19|
+-------+-----+
only showing top 10 rows



In [9]:
# Count the number of times a user who rated and show top 10
df_user_count = df.groupby('userId').count()
df_user_count.sort('count', ascending = False).show(10)

+------+-----+
|userId|count|
+------+-----+
|     6|   57|
|    14|   57|
|    11|   56|
|    22|   56|
|     4|   55|
|    12|   55|
|     7|   54|
|     9|   53|
|    24|   52|
|    23|   52|
+------+-----+
only showing top 10 rows



### 2. Split dataset into train and test. Try 2 different combinations for e.g. (60/40, 70/30, 75/25 and 80/20). (Train your model and use collaborative filtering approach on 70 percent of your data and test with the other 30 percent and so on). Show your code and output.

In [11]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

#### 80/20 combination 

In [12]:
# Create train and test dataset, here we use 0.8 / 0.2
(training1, test1) = df.randomSplit([0.8, 0.2])

In [144]:
# Build the recommendation model using ALS on the training data
als = ALS(userCol = "userId", itemCol = "movieId", ratingCol = "rating", 
          coldStartStrategy = "drop")

In [145]:
#Fit the als model to training data
model1 = als.fit(training1)

# Get the predictions
predictions1 = model1.transform(test1)

#### 70/30 combination 

In [153]:
# Create train and test dataset, here we use 0.7 / 0.3
(training2, test2) = df.randomSplit([0.7, 0.3])

In [154]:
#Fit the als model to training data
model2 = als.fit(training2)

# Get the predictions
predictions2 = model2.transform(test2)

### 3. Explain MSE, RMSE and MAE. Compare and evaluate both of your models with evaluation metrics (RMSE or MAE), show your code and print your results. Describe which one works better and why?

In [147]:
# RMSE
evaluator_rmse = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
# MSE
evaluator_mse = RegressionEvaluator(metricName="mse", labelCol="rating",
                                predictionCol="prediction")
# MAE
evaluator_mae = RegressionEvaluator(metricName="mae", labelCol="rating",
                                predictionCol="prediction")

#### 80/20 combination

In [148]:
# Evaluate the model by computing the RMSE, MSE, MAE on the test data
rmse1 = evaluator_rmse.evaluate(predictions1)
mse1 = evaluator_mse.evaluate(predictions1)
mae1 = evaluator_mae.evaluate(predictions1)

In [149]:
print("Root-mean-square error = " + str(rmse1))
print("Mean-square error = " + str(mse1))
print("Mean-absolute error = " + str(mae1))

Root-mean-square error = 0.9099785118595655
Mean-square error = 0.8280608920461494
Mean-absolute error = 0.6348494553861961


#### 70/30 combination

In [155]:
# Evaluate the model by computing the RMSE, MSE, MAE on the test data
rmse2 = evaluator_rmse.evaluate(predictions2)
mse2 = evaluator_mse.evaluate(predictions2)
mae2 = evaluator_mae.evaluate(predictions2)

In [156]:
print("Root-mean-square error = " + str(rmse2))
print("Mean-square error = " + str(mse2))
print("Mean-absolute error = " + str(mae2))

Root-mean-square error = 1.0794408792156327
Mean-square error = 1.165192611721818
Mean-absolute error = 0.7424439422650152


#### Which one of the two splits you tried in B-2 worked better?

From the output, we can see the RMSE for 80/20 combination is around 0.91
, the MSE for 80/20 combination is around 0.83 while the MAE for 80/20 combination is around0.64. For 70/30 combination, the RMSE is around 1.08, the MSE is around 1.17, and the MAE is around 0.74.

By comparing the result of RMSE, MSE, and MAE for two different splits, we noticed that all three kinds of error in 80/20 combination are samller than those in 70/30 combination, therefore, the 80/20 combination  works better.



### 4. Now tune the parameters of your algorithm to get the best set of parameters. Explain different parameters of the algorithm which you have used for tuning your algorithm. Evaluate all your models again. Show your code with best values and output.

In [33]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator, TrainValidationSplit

In [36]:
# initialize the ALS model
als = ALS(userCol='userId', itemCol='movieId', ratingCol='rating', coldStartStrategy='drop')

There are many hyperparameters of ALS such as rank, maxIter, regParam, numItemBlocks, numUserBlocks. 

Here I choose `rank`, `maxIter`, and `regParam` to tune.

`rank` is the number of latent factors in the model and the default value is 10 so I set the grid [5, 10, 50].

 `maxIter` the maximum number of iterations to run and the default value is 10 so I set the grid [5, 10, 15].
 
`regParam` is the regularization parameter in ALS and the default value is 1.0 so I set the grid [0.01, 0.1, 1.0].



In [48]:
# create the parameter grid
parameters = ParamGridBuilder().addGrid(als.regParam, [.01, .1, 1.0]
                              ).addGrid(als.rank, [5, 10, 50]
                              ).addGrid(als.maxIter, [5, 10, 15]          
                              ).build()

In [55]:
print ("Number of models to be tested: ", len(parameters))

Number of models to be tested:  27


In [49]:
#Build cross validator
cv = CrossValidator(estimator = als, estimatorParamMaps = parameters,
                    evaluator = eval, numFolds = 3)

#Fit model to training data using 80/20 comb
model = cv.fit(training1)

#Extract best model from the cv model above
best_model = model.bestModel

In [51]:
# Check the best model parameters
print("**Best Model**")
# Print "Rank"
print("  Rank:", best_model._java_obj.parent().getRank())
# Print "MaxIter"
print("  MaxIter:", best_model._java_obj.parent().getMaxIter())
# Print "RegParam"
print("  RegParam:", best_model._java_obj.parent().getRegParam())

**Best Model**
  Rank: 50
  MaxIter: 15
  RegParam: 0.1


After feed 27 combs of parameters into the crossvalidator, we find the best model where `rank` is  50, `maxIter` is 15, and `regParam` is 0.1.

In [52]:
#Use model evaluator as RMSE
eval = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")

predictions = best_model.transform(test1)
rmse = eval.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 0.8944824479975415


Based on the best model, the RMSE is around 0.89 which means that on average the model predicts 0.89 above or below values of the original ratings matrix. It is smaller than the RMSE we got in B-3 which makes sense as it is the best model after tuning hyperparameters.

### 5. Calculate top 10 movies recommendations for user id 10 and user id 14. Show your code and output.

In [ ]:
# Generate n Recommendations for all users
recommendations = best_model.recommendForAllUsers(50)

In [ ]:
# Convert recommendations into interpretable format
recommendations = recommendations\
    .withColumn("rec_exp", explode("recommendations"))\
    .select('userId', col("rec_exp.movieId"), col("rec_exp.rating"))

#### User ID 10

In [129]:
# filter the dataframe only containing user id 10
df_10 = df.filter('userId = 10')

# create a movie id list that already rated by user
movieIdlist_10 = [int(row['movieId']) for row in df_10.collect()]

In [130]:
# Recommendations for user id 10
Rec_id10 = recommendations.filter('userId = 10')

In [132]:
# Remove movies that already rated by user in the recommendation list
movieRec_id10 = Rec_id10.filter(Rec_id10.movieId.isin(movieIdlist_10)==False)

# show top 10 movies recommendations for user id 10
movieRec_id10.limit(10).show()

+------+-------+---------+
|userId|movieId|   rating|
+------+-------+---------+
|    10|     92|2.8280194|
|    10|     62|2.6331937|
|    10|     12| 2.556482|
|    10|     81|2.3621356|
|    10|     87|2.2791731|
|    10|     70|2.2654696|
|    10|     95|2.2651637|
|    10|     82|2.1536677|
|    10|     29| 2.057789|
|    10|     91|2.0360448|
+------+-------+---------+



#### User ID 14

In [133]:
# filter the dataframe only containing user id 14
df_14 = df.filter('userId = 14')

# create a movie id list that already rated by user
movieIdlist_14 = [int(row['movieId']) for row in df_14.collect()]

In [136]:
# Recommendations for user id 14
Rec_id14 = recommendations.filter('userId = 14')

In [137]:
# Remove movies that already rated by user in the recommendation list
movieRec_id14 = Rec_id14.filter(Rec_id14.movieId.isin(movieIdlist_14)==False)

# show top 10 movies recommendations for user id 14
movieRec_id14.limit(10).show()

+------+-------+---------+
|userId|movieId|   rating|
+------+-------+---------+
|    14|     30|3.3439934|
|    14|     85|3.2353022|
|    14|      2|2.6936684|
|    14|     74|2.6150405|
|    14|     43|2.6082118|
|    14|     90|2.5717528|
|    14|     77|2.5677755|
|    14|     70|2.4696212|
|    14|     58|2.3936088|
|    14|     87|2.2354074|
+------+-------+---------+

